## 实验：Pandas 代码优化

### 本地环境配置

In [1]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['font.family'] = 'sans-serif'    # 用来正常显示中文
plt.rcParams['font.sans-serif'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False   # 设置正常显示符号

# 设置输入输出路径
import os
base_path = os.environ.get("BASE_PATH",'../data/')
data_path = os.path.join(base_path + "lab3/")
result_path = "result/"
os.makedirs(result_path, exist_ok=True)

# 忽略第三方支援库更新兼容性提示
import warnings
warnings.simplefilter('ignore') 

### 指令格式

请作为一个代码优化器，帮我优化以下 Pandas 代码 ——

**<font color="#0000dd">Python 代码</font>**

### 指令示例

请作为一个代码优化器，帮我优化以下 Pandas 代码 ——
```python
import pandas as pd

df = pd.read_csv(data_path + "titles.csv") 
df_bad = df.query("runtime > 30 & type == 'SHOW'") 
df_bad ["score"] = df_bad[["imdb_score", "tmdb_score"]].sum(axis=1) 
df_bad = df_bad[["seasons", "score"]] 
df_bad = df_bad.groupby("seasons").agg(["count", "mean"]) 
df_bad = df_bad.droplevel(axis=1, level=0) 
df_bad = df_bad.query("count > 10") 

df_bad.head()
```

<img src="./img/3-5.png" width=80%>

### 原始代码

大多数 pandas 从业者在学习 pandas 的数据处理时，都是通过对 DataFrames 进行连续的变更，作为一系列不同的、逐行的操作。对 pandas DataFrames 进行过度的变更会导致一些问题，有几个原因。

1. 它通过创建全局变量来浪费内存（如果你在每一步都创建一个不同命名的 DataFrame，那就更是如此）
2. 代码很繁琐，难以阅读
3. 容易出现错误--特别是在笔记本中，数据操作步骤的顺序可能不会被严格执行

与其依次变更一个 pandas DataFrame，更好的方法是通过链式的 pandas 方法对其进行转换

In [2]:
import pandas as pd

df = pd.read_csv(data_path + "titles.csv") 
df_bad = df.query("runtime > 30 & type == 'SHOW'") 
df_bad ["score"] = df_bad[["imdb_score", "tmdb_score"]].sum(axis=1) 
df_bad = df_bad[["seasons", "score"]] 
df_bad = df_bad.groupby("seasons").agg(["count", "mean"]) 
df_bad = df_bad.droplevel(axis=1, level=0) 
df_bad = df_bad.query("count > 10") 

df_bad.head()

,count,mean
seasons,,
1.0,876,13.140103
2.0,206,13.974840
3.0,84,14.419810
4.0,44,15.059500
5.0,38,15.132395


### ChatGPT 代码测试

In [3]:
df = pd.read_csv(data_path + "titles.csv") 
df_bad = (df.query("runtime > 30 & type == 'SHOW'")
             .assign(score = df[["imdb_score", "tmdb_score"]].sum(axis=1))
             .filter(["seasons", "score"])
             .groupby("seasons")
             .agg(["count", "mean"])
             .droplevel(axis=1, level=0)
             .query("count > 10")
         )

df_bad.head()

,count,mean
seasons,,
1.0,876,13.140103
2.0,206,13.974840
3.0,84,14.419810
4.0,44,15.059500
5.0,38,15.132395


原始代码来源：https://www.aidancooper.co.uk/pandas-anti-patterns/